In [51]:
import andes
import ams
import cvxpy as cp
import numpy as np

# Load and formulate ams case

In [52]:
ssp = ams.load(ams.get_case('ieee14/ieee14_uced.xlsx'))

ssp.LDOPF.run()

LDOPF solved as optimal in 0.0114 seconds with exit code 0.


True

In [53]:
ssp.LDOPF.om

In [54]:
ssp.LDOPF.om.vars

OrderedDict([('pg', Variable((5,))),
             ('pn', Variable((14,))),
             ('qg', Variable((5,))),
             ('qn', Variable((14,))),
             ('vsq', Variable((14,))),
             ('pl', Variable((20,))),
             ('ql', Variable((20,)))])

In [55]:
cvx_mdl = ssp.LDOPF.om.mdl

cvx_mdl.solve()

69.55849510545293

In [56]:
ssp.LDOPF.om.vars['pg'].value

array([0.49999943, 0.49999947, 0.36850084, 0.36850084, 0.49999943])

In [57]:
ssp.LDOPF.om.pg.value

array([0.49999943, 0.49999947, 0.36850084, 0.36850084, 0.49999943])

# Modify ams cvxpy model

### define new variables

In [58]:
# define new variables

lb = np.array([0, 0])
ub = np.array([1, 1])

x = cp.Variable(2, name="x")

# x = cp.Variable(2, name='x', lb=np.array([0, 0]), ub=np.array([1, 1]))

### modify constraints

In [59]:
# define new constraints

constraints_new = [x >= lb, x <= ub]

In [60]:
# extract constraints from ams model

constriants_ams = [ssp.LDOPF.om.constrs[cname] for cname in ssp.LDOPF.om.constrs.keys()]
constriants_ams

[Inequality(Constant(CONSTANT, NONNEGATIVE, (5,))),
 Inequality(Variable((5,))),
 Inequality(Constant(CONSTANT, NONPOSITIVE, (5,))),
 Inequality(Variable((5,))),
 Equality(Expression(AFFINE, UNKNOWN, ()), Constant(CONSTANT, ZERO, ())),
 Equality(Expression(AFFINE, UNKNOWN, (14,)), Constant(CONSTANT, ZERO, ())),
 Inequality(Expression(AFFINE, UNKNOWN, (20,))),
 Inequality(Expression(AFFINE, UNKNOWN, (20,))),
 Inequality(Expression(AFFINE, UNKNOWN, (14,))),
 Inequality(Expression(AFFINE, UNKNOWN, (14,))),
 Equality(Expression(AFFINE, UNKNOWN, (14,)), Constant(CONSTANT, ZERO, ())),
 Equality(Expression(AFFINE, UNKNOWN, ()), Constant(CONSTANT, ZERO, ())),
 Equality(Expression(AFFINE, UNKNOWN, (20,)), Constant(CONSTANT, ZERO, ()))]

In [61]:
# formulated new constraints

constraints_new += constriants_ams
constraints_new

[Inequality(Constant(CONSTANT, ZERO, (2,))),
 Inequality(Variable((2,))),
 Inequality(Constant(CONSTANT, NONNEGATIVE, (5,))),
 Inequality(Variable((5,))),
 Inequality(Constant(CONSTANT, NONPOSITIVE, (5,))),
 Inequality(Variable((5,))),
 Equality(Expression(AFFINE, UNKNOWN, ()), Constant(CONSTANT, ZERO, ())),
 Equality(Expression(AFFINE, UNKNOWN, (14,)), Constant(CONSTANT, ZERO, ())),
 Inequality(Expression(AFFINE, UNKNOWN, (20,))),
 Inequality(Expression(AFFINE, UNKNOWN, (20,))),
 Inequality(Expression(AFFINE, UNKNOWN, (14,))),
 Inequality(Expression(AFFINE, UNKNOWN, (14,))),
 Equality(Expression(AFFINE, UNKNOWN, (14,)), Constant(CONSTANT, ZERO, ())),
 Equality(Expression(AFFINE, UNKNOWN, ()), Constant(CONSTANT, ZERO, ())),
 Equality(Expression(AFFINE, UNKNOWN, (20,)), Constant(CONSTANT, ZERO, ()))]

### define new objective function

In [62]:
obj = cp.Minimize(cp.sum(x))

In [63]:
x

Variable((2,))

In [64]:
np.array([1, 2]).shape

(2,)

In [65]:
temp = x @ np.array([1, 2])
temp

Expression(AFFINE, UNKNOWN, ())

In [66]:
obj = cp.Minimize(x @ np.array([1, 2]))

### formulate new cvxpy problem

In [67]:
prob = cp.Problem(obj, constraints_new)

In [68]:
prob.solve()

1.5578968827277052e-11

# Check optimization results

In [69]:
# optimization status

if prob.status == cp.OPTIMAL:
    print("Optimal solution found.")
elif prob.status == cp.INFEASIBLE:
    print("The problem is infeasible.")
elif prob.status == cp.UNBOUNDED:
    print("The problem is unbounded.")
else:
    print("Solver did not converge or encountered an issue.")

Optimal solution found.


In [70]:
# objective value

optimal_objective_value = prob.value

print("Optimal Objective Value:", optimal_objective_value)

Optimal Objective Value: 1.5578968827277052e-11


In [71]:
x.value

array([0., 0.])

In [72]:
# decision variables values

# pg_optimal = prob.pg.value

# print("Optimal values of 'pg':", pg_optimal)

In [73]:
# decision variables values

pg_optimal = ssp.LDOPF.om.pg.value

print("Optimal values of 'pg':", pg_optimal)

Optimal values of 'pg': [0.27023518 0.27023518 0.27023518 0.27023518 1.15605926]


In [74]:
ssp.LDOPF.om.vars['pg']

Variable((5,))

In [75]:
ssp.LDOPF.om.pg.value

array([0.27023518, 0.27023518, 0.27023518, 0.27023518, 1.15605926])